In [244]:
# !pip install langchain_experimental
# langchain_openai
# !pip install rank_bm25
# !pip install yandexcloud

In [9]:
import warnings 
warnings.filterwarnings('ignore')

In [10]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from langchain_community.document_loaders import JSONLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Qdrant
# from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from ragatouille import RAGPretrainedModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Tuple, Optional
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS


In [11]:
with open('train_Tinkoff/dataset.json', 'r') as f:
    f = json.loads(f.read())

for i in range(len(f['data'])):
    f['data'][i]['id_doc'] = i

documents_raw_dict = {}
for i in range(len(f['data'])):
    documents_raw_dict[f['data'][i]['id_doc']] = f['data'][i]['description']

with open('dataset.json', 'w') as fout:
    json.dump(f['data'] , fout)

def metadata_func(content: dict, metadata: dict) -> dict:
    metadata['id_doc'] = content.get('id_doc')
    metadata["title"] = content.get("title")
    metadata["parent_title"] = content.get("parent_title")
    metadata["business_line_id"] = content.get("business_line_id")
    metadata["direction"] = content.get("direction")
    metadata["product"] = content.get("product")
    metadata["type"] = content.get("type")
    metadata["url"] = content.get("url")
    return metadata

loader = JSONLoader(
    file_path='./dataset.json',
    jq_schema='.[]',
    content_key='description',
    metadata_func=metadata_func)

documents = loader.load()[:100]

for i in range(len(documents)):
    documents[i].page_content = documents[i].metadata['title'] + ' ' + documents[i].page_content

In [15]:
f['data'][0]

{'id': '3e75f57f-3f6f-4eaf-ae5d-ec6586d201f1',
 'title': 'Как посмотреть лимиты счета?',
 'url': 'https://www.tinkoff.ru/business/help/business-payout/jump/about/summary/?card=q1',
 'source': 'web',
 'business_line_id': 'business',
 'direction': 'business-payout',
 'product': 'jump',
 'type': 'card',
 'description': 'Вы можете посмотреть лимиты по счету на месяц, по карте на месяц и на разовую выплату. Для этого: Откройте раздел «Сводка». В разделе «Счета» найдите нужное вам юридическое лицо и нажмите на него. В открывшемся окне будут представлены все расчетные счета, к которым есть доступ с этого юридического лица. Найдите нужный вам счет.',
 'parent_title': 'Сводка',
 'parent_url': 'https://www.tinkoff.ru/business/help/business-payout/jump/about/summary/',
 'id_doc': 0}

In [182]:
# def split_documents(
#     chunk_size: int,
#     knowledge_base: List[LangchainDocument],
# ) -> List[LangchainDocument]:
#     """
#     Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
#     """
    
#     # We use a hierarchical list of separators specifically tailored for splitting Markdown documents
#     # This list is taken from LangChain's MarkdownTextSplitter class
#     MARKDOWN_SEPARATORS = [
#         "\n#{1,6} ",
#         "```\n",
#         "\n\\*\\*\\*+\n",
#         "\n---+\n",
#         "\n___+\n",
#         "\n\n",
#         "\n",
#         " ",
#         "",
#     ]
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=int(chunk_size / 10),
#         add_start_index=True,
#         strip_whitespace=True,
#         separators=MARKDOWN_SEPARATORS,
#     )

#     docs_processed = []
#     for doc in knowledge_base:
#         docs_processed += text_splitter.split_documents([doc])

#     # Remove duplicates
#     unique_texts = {}
#     docs_processed_unique = []
#     for doc in docs_processed:
#         if doc.page_content not in unique_texts:
#             unique_texts[doc.page_content] = True
#             docs_processed_unique.append(doc)

#     return docs_processed_unique


# docs_processed = split_documents(
#     1000,  # We choose a chunk size adapted to our model
#     documents,
# )

In [183]:
def split_documents(
    knowledge_base: List[LangchainDocument],
    embeddings
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    
    # We use a hierarchical list of separators specifically tailored for splitting Markdown documents
    # This list is taken from LangChain's MarkdownTextSplitter class
    MARKDOWN_SEPARATORS = [
        "\n#{1,6} ",
        "```\n",
        "\n\\*\\*\\*+\n",
        "\n---+\n",
        "\n___+\n",
        "\n\n",
        "\n",
        " ",
        "",
    ]

    text_splitter = SemanticChunker(
        embeddings, breakpoint_threshold_type="percentile"
    )

    docs_processed = []
    for doc in tqdm(knowledge_base):
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed


embeddings = OllamaEmbeddings(model='aya:8b')

docs_processed = split_documents(
    documents,
    embeddings
)

100%|█████████████████████████████████████████| 100/100 [01:42<00:00,  1.02s/it]


In [184]:
len(docs_processed)

182

In [193]:
docs_processed[0]

Document(page_content='Как посмотреть лимиты счета?', metadata={'source': '/Users/hare/code2/tinkoff/dataset.json', 'seq_num': 1, 'id_doc': 0, 'title': 'Как посмотреть лимиты счета?', 'parent_title': 'Сводка', 'business_line_id': 'business', 'direction': 'business-payout', 'product': 'jump', 'type': 'card', 'url': 'https://www.tinkoff.ru/business/help/business-payout/jump/about/summary/?card=q1'})

In [200]:
KNOWLEDGE_VECTOR_DATABASE.similarity_search_with_score('dfdf')[0]

(Document(page_content='Как добавить пользователя в личный кабинет?', metadata={'source': '/Users/hare/code2/tinkoff/dataset.json', 'seq_num': 28, 'id_doc': 27, 'title': 'Как добавить пользователя в личный кабинет?', 'parent_title': 'Настройки', 'business_line_id': 'business', 'direction': 'business-payout', 'product': 'jump', 'type': 'card', 'url': 'https://www.tinkoff.ru/business/help/business-payout/jump/about/settings/?card=q1', '_id': 'c902f916c2964ee595c4db4186ea82b8', '_collection_name': 'my_documents'}),
 0.120165473298223)

In [212]:
from transformers import Pipeline
from typing import Tuple, Optional, List


def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: None,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 5,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:

    llm = ChatOllama(model="aya:8b")

    # Give analogue questions
    questions = fusion_questions(question, llm, 4)
    print(questions)
    print()

    # Gather documents with retriever
    print("=> Retrieving documents...")

    relevant = []
    scores = []
    for question in questions:
        rels = knowledge_index.similarity_search_with_score(query=question, k=num_retrieved_docs)
        for rel in rels:
            relevant.append(rel[0])
            scores.append(rel[1])
        # relevant.extend(ensemble_retriever.invoke(question, k=num_retrieved_docs))

    # Remove duplicates
    unique_texts = {}
    relevant_unique = []
    scores_unique = []
    for i, doc in enumerate(relevant):
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            relevant_unique.append(doc)
            scores_unique.append(scores[i])
    print(f"=> Unique documents: {len(relevant_unique)}")

    relevant_docs = [doc for i, doc in enumerate(relevant_unique) if scores_unique[i] > np.quantile(scores_unique, 0.9)]
    
    # relevant_docs = [doc.page_content for doc in relevant_unique]  # Keep only the text

    # print(relevant_unique)
    
    # # Optionally rerank results
    # reranker_k = min(10, len(relevant_docs))
    # if reranker:
    #     print("=> Reranking documents...")
    #     relevant_docs = reranker.rerank(question, relevant_docs, k=reranker_k)

    #     scores = [doc["score"] for doc in relevant_docs]
    #     print(np.quantile(scores, 0.9))
        
    #     relevant_docs = [doc['content'] for doc in relevant_docs if doc['score'] > np.quantile(scores, 0.9)]
    #     print(relevant_docs)

    # relevant_unique = [doc for doc in relevant_unique if doc.page_content in relevant_docs]

    # relevant_unique_temp = []
    # for content in relevant_docs:
    #     for doc in relevant_unique:
    #         if content == doc.page_content:
    #             relevant_unique_temp.append(doc)

    # print(relevant_unique_temp)
    
    # Remove duplicates
    unique_urls = {}
    relevant_unique_ = []
    for doc in relevant_docs:
        if doc.metadata['url']not in unique_urls:
            unique_urls[doc.metadata['url']] = True
            relevant_unique_.append(doc)

    relevant_docs = [documents_raw_dict[x.metadata['id_doc']] for x in relevant_unique_]

    print(len(relevant_docs))
    print(relevant_docs)

    context = "\nИзвлеченные документы:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    messages = [
        SystemMessage(
            content="""
            Ты - интеллектуальный ассистент, который помогает пользователям находить информацию и отвечать на их вопросы. 
            Используй всю информацию из контекста.
            Пиши без лишней информации.
            Если ответ не может быть выведен из контекста, не отвечайте."""
        ),
        HumanMessage(
            content=f"""Контекст: {context}
            ---
            Вопрос, на который нужно ответить: {question}"""
                    ),
    ]
    # print(f"Контекст: {context} --- Вопрос, на который нужно ответить: {question}")
    # Redact an answer
    print("=> Generating answer...")
    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({})

    answer += f'\n\nБолее подробную информацию вы можете найти по ссылкам: '
    for i, x in enumerate(relevant_unique_):
        if i != len(relevant_unique_)-1:
            answer += x.metadata['url'] + ', '
        else:
            answer += x.metadata['url']

    return answer, [x.metadata['url'] for x in relevant_unique_]



In [204]:
from transformers import Pipeline
from typing import Tuple, Optional, List

def fusion_questions(
    question: str,
    llm: Pipeline,
    num_questions: int = 2,
) -> List[str]:

    llm = ChatOllama(model="aya:8b")

    questions = [question]

    for _ in range(num_questions):
        messages = [
            SystemMessage(
                content="""Ты интеллектуальный помощник, улучшающий чатбот техподдержки. 
                Твоя задача генирировать альтернативные формулировки вопроса пользователя.
                Пиши только сгенерированный запрос, без другого текста."""
            ),
            HumanMessage(
                content=f"""Вопрос, который нужно переформулировать: {question}"""),
        ]
    
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | llm | StrOutputParser()
        answer = chain.invoke({})

        questions.append(answer)

    return questions

In [196]:
embeddings = OllamaEmbeddings(model='aya:8b')
llm = ChatOllama(model="aya:8b")
# RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
RERANKER = RAGPretrainedModel.from_pretrained("colbert/")

KNOWLEDGE_VECTOR_DATABASE = Qdrant.from_documents(
    docs_processed,
    embeddings,
    path="./local_qdrant",
    collection_name="my_documents",
)

# KNOWLEDGE_VECTOR_DATABASE = Qdrant.from_existing_collection(embeddings, path='./local_qdrant', collection_name="my_documents")

# # initialize the bm25 retriever and faiss retriever
# bm25_retriever = BM25Retriever.from_documents(
#     docs_processed
# )
# bm25_retriever.k = 5

# with open('bm25result', 'wb') as bm25result_file:
#     pickle.dump(bm25_retriever, bm25result_file)
# # with open('bm25result', 'rb') as bm25result_file:
# #     bm25_retriever = pickle.load(bm25result_file)

# embedding = OllamaEmbeddings(model='aya:8b')
# faiss_vectorstore = FAISS.from_documents(
#     docs_processed, embedding
# )
# with open('faissresult', 'wb') as faissresult_file:
#     pickle.dump(faiss_vectorstore, faissresult_file)
# # with open('faissresult', 'rb') as faissresult_file:
# #     faiss_vectorstore = pickle.load(faissresult_file)

# faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 5})

# # initialize the ensemble retriever
# ensemble_retriever = EnsembleRetriever(
#     retrievers=[
#         # bm25_retriever, 
#                 faiss_retriever], weights=[0.5, 0.5]
# )

In [216]:
# question = "Как настроить интеграцию с кассой АТОЛ?"
question = "Как посмотреть лимиты счета?"

answer, relevant_docs = answer_with_rag(
    question, llm, KNOWLEDGE_VECTOR_DATABASE, 
    # reranker=RERANKER,
    num_docs_final=5
)

['Как посмотреть лимиты счета?', 'Каким образом я могу узнать лимиты своего счета?', 'Как проверить лимиты моего аккаунта?', 'Как проверить лимиты моего аккаунта?', '"Как проверить лимиты моего аккаунта?"']

=> Retrieving documents...
=> Unique documents: 7
1
['Вы можете посмотреть лимиты по счету на месяц, по карте на месяц и на разовую выплату. Для этого: Откройте раздел «Сводка». В разделе «Счета» найдите нужное вам юридическое лицо и нажмите на него. В открывшемся окне будут представлены все расчетные счета, к которым есть доступ с этого юридического лица. Найдите нужный вам счет.']
=> Generating answer...


In [217]:
print("==================================Answer==================================")
print(f"{answer}")
# print("\n==================================Source docs==================================")
# for i, doc in enumerate(relevant_docs):
#     print(f"Document {i}------------------------------------------------------------")
#     print(doc)

==================================Answer==================================
Для проверки лимитов своего аккаунта необходимо выполнить следующие шаги: 

- Откройте раздел "Сводка" в своем персональном кабинете. 
- В разделе "Счета" выберите нужное юридическое лицо, используя соответствующую опцию. 
- После выбора юридического лица вы увидите список всех счетов, связанных с этим лицом. 
- Найдите нужный счет в списке и откройте его подробную информацию. 
- В информации о счете будут представлены сведения о лимитах, доступных для этого конкретного аккаунта. 

Эти инструкции позволяют быстро проверить текущие лимиты вашего аккаунта и убедиться, что вы не превысили их.

Более подробную информацию вы можете найти по ссылкам: https://www.tinkoff.ru/business/help/business-payout/jump/about/summary/?card=q1


In [1]:
from rag import ChatBotRAG

# Example usage
data_path = 'train_Tinkoff/dataset.json'
vector_db_path = './local_qdrant'
embeddings_model = 'aya:8b'
llm_name = 'aya:8b'
light_llm_name = 'aya:8b'

/Users/hare/opt/anaconda3/envs/whisper/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chatbot = ChatBotRAG(data_path, vector_db_path, embeddings_model, llm_name, light_llm_name)

In [3]:
chatbot.load_data()

In [ ]:
docs_processed = chatbot.split_documents()

  6%|██▍                                   | 209/3237 [04:05<1:27:00,  1.72s/it]

In [ ]:
chatbot.create_vector_database(docs_processed)

In [ ]:
question = "Как настроить интеграцию с кассой АТОЛ?"
answer, url_links = chatbot.answer_with_rag(question, num_docs_final=5)

print("==================================Answer==================================")
print(f"{answer}")
print("\n==================================Source docs==================================")
for i, url in enumerate(url_links):
    print(f"{i}) {url}\n")

In [ ]:
print(answer)
print()
print(url_links)